## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-01-20-51-59 +0000,nyt,Guinea’s Coup Leader Wins Election After Barri...,https://www.nytimes.com/2026/01/01/world/afric...
1,2026-01-01-20-48-28 +0000,nyt,Live Updates: Dozens Confirmed Dead in Fire at...,https://www.nytimes.com/live/2026/01/01/world/...
2,2026-01-01-20-43-33 +0000,wapo,New York City Mayor Zohran Mamdani is sworn in...,https://www.washingtonpost.com/nation/2026/01/...
3,2026-01-01-20-42-25 +0000,bbc,"Around 40 dead in Swiss ski resort bar fire, p...",https://www.bbc.com/news/articles/cx2pmyyeg78o...
4,2026-01-01-20-42-03 +0000,nyt,Fire in Swiss Alps Leaves Dozens of New Year’s...,https://www.nytimes.com/2026/01/01/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2395/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,new,44
15,year,41
61,trump,36
13,fire,17
17,swiss,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
101,2026-01-01-11-13-29 +0000,cbc,"Around 40 dead, more than 100 injured in blaze...",https://www.cbc.ca/news/world/bar-fire-switzer...,159
157,2026-01-01-02-19-53 +0000,nypost,"Trump calls for ‘peace on Earth,’ vows to ‘get...",https://nypost.com/2025/12/31/us-news/trump-ca...,156
14,2026-01-01-20-08-08 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...,155
134,2026-01-01-07-42-14 +0000,nypost,"40 people dead, 100 hurt after fire rips throu...",https://nypost.com/2026/01/01/world-news/swiss...,155
85,2026-01-01-13-52-12 +0000,nypost,Horrific video shows the moment deadly New Yea...,https://nypost.com/2026/01/01/world-news/new-y...,154


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
101,159,2026-01-01-11-13-29 +0000,cbc,"Around 40 dead, more than 100 injured in blaze...",https://www.cbc.ca/news/world/bar-fire-switzer...
70,76,2026-01-01-15-40-43 +0000,nyt,Trump Must Return Command of California Nation...,https://www.nytimes.com/2025/12/31/us/trump-na...
16,53,2026-01-01-20-02-35 +0000,nypost,‘Tax the rich’ chant breaks out as Bernie Sand...,https://nypost.com/2026/01/01/us-news/tax-the-...
155,39,2026-01-01-02-29-38 +0000,nyt,CI.A. Says Ukraine Did Not Target Putin’s Home...,https://www.nytimes.com/2025/12/31/us/politics...
184,33,2025-12-31-23-40-15 +0000,latimes,LAFD leaders tried to cover up Palisades fire ...,https://www.latimes.com/california/story/2025-...
17,31,2026-01-01-19-55-28 +0000,nypost,Son of late New York musician was ready to com...,https://nypost.com/2026/01/01/us-news/grieving...
159,31,2026-01-01-02-10-54 +0000,latimes,Puppy found in trash compactor in Gardena. Pol...,https://www.latimes.com/california/story/2025-...
94,31,2026-01-01-12-48-00 +0000,wsj,President Trump delayed by a year tariff incre...,https://www.wsj.com/economy/trade/trump-rolls-...
157,29,2026-01-01-02-19-53 +0000,nypost,"Trump calls for ‘peace on Earth,’ vows to ‘get...",https://nypost.com/2025/12/31/us-news/trump-ca...
33,28,2026-01-01-18-37-28 +0000,nypost,Farming tycoon accused of killing wife could p...,https://nypost.com/2026/01/01/us-news/judge-se...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
